<a href="https://colab.research.google.com/github/vafter341ew/COLAB/blob/main/lab6_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, mean, expr

spark = SparkSession.builder.appName("HealthSurvey").getOrCreate()

reverse_items = spark.read.csv("data/ReverseCodingItems_v2.csv", header=True, inferSchema=True)
reverse_list = [row["Question Identifier"] for row in reverse_items.filter(col("Needs Reverse Coding?") == "Yes").collect()]

coding_map = {"Strongly Disagree": 1, "Somewhat Disagree": 2, "Neither Agree nor Disagree": 3, "Somewhat Agree": 4, "Strongly Agree": 5}
reverse_map = {"Strongly Disagree": 5, "Somewhat Disagree": 4, "Neither Agree nor Disagree": 3, "Somewhat Agree": 2, "Strongly Agree": 1}
factor_map = {"Physical Health": "F1", "Mental Health": "F2", "Social Health": "F3", "Environmental Health": "F4", "Spiritual Health": "F5", "Intellectual Health": "F6"}

result = (
    spark.read.csv("data/health_survey.csv", header=True, inferSchema=True)
    .join(reverse_items.select(col("Question Identifier"), col("Question Type")),
          expr("stack(6, " + ", ".join([f"'{q}', {q}" for q in [c for c in spark.read.csv("data/health_survey.csv", header=True).columns if c != "RespondentID"]]) + ") as (Question, Response)"),
          col("Question") == col("Question Identifier"))
    .withColumn("Recoded Value",
                when(col("Question").isin(reverse_list),
                     expr(" ".join([f"WHEN Response = '{k}' THEN {v}" for k, v in reverse_map.items()])))
                .otherwise(expr(" ".join([f"WHEN Response = '{k}' THEN {v}" for k, v in coding_map.items()]))))
    .withColumn("Factor", expr(" ".join([f"WHEN `Question Type` = '{k}' THEN '{v}'" for k, v in factor_map.items()])))
    .groupBy("RespondentID", "Factor").agg(mean("Recoded Value").alias("Mean Score"))
    .groupBy("RespondentID").pivot("Factor").agg(expr("first(`Mean Score`)"))
)

result.write.csv("data/health_survey_summary.csv", header=True, mode="overwrite")

spark.stop()

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/content/data/ReverseCodingItems_v2.csv.